# BGS MODELS

In [1]:
library(tidyverse)
library(cowplot)
library(Rcpp)
library(purrr)
library(tidyr)
theme_set(cowplot::theme_cowplot())
library(patchwork)
options(repr.plot.width = 10, repr.plot.height = 7, repr.plot.res = 200)

#Calculates G
cppFunction(
  'NumericVector get_G(double U, double sh, double P, NumericVector fd_i, NumericVector M_1, NumericVector M_2) {

  int n = M_1.size();
  double G_k;
  double M_i;
  NumericVector G_i(n);

  for(int i = 0; i < n; ++i) {
    M_i =  (M_1[i] + M_2[i]) / 2;
    for(int k = 0; k < n; ++k){
      G_k = U * fd_i[k] * sh / (2 * (sh + P*std::abs(M_i - M_1[k]))*(sh + P*std::abs(M_i - M_2[k])));
      G_i[i] += G_k;
    }
  }
  return G_i;
}'
)


#deterministic model components
theta_full <- function(theta, G_i, alpha, fd_i, rbp_i){
  theta / (1/exp(-G_i) + alpha * fd_i / rbp_i)
}

theta_noBGS <- function(theta, G_i, alpha, fd_i, rbp_i){
  theta / (1 + alpha * fd_i / rbp_i)
} 

theta_noHH <- function(theta, G_i, alpha, fd_i, rbp_i){
  theta / (1/exp(-G_i))
} 

theta_intercept <- function(theta, G_i, alpha, fd_i, rbp_i){
  theta
}

#log likelihood equations to optimize
LL_full <- function(pi_obs, par, G_i, fd_i, rbp_i, mode = "beta"){
  
  #rescaled parameters to make optimization easier
  theta <- 1/exp(par[1])
  alpha <- 1/exp(par[2])
  sigma <- 1/exp(par[3])
  
  pi_pred <- theta_full(theta, G_i, alpha, fd_i, rbp_i)
  
  if(mode == "beta"){
    -sum(dbeta(x = pi_obs, shape1 = pi_pred * sigma, shape2 = (1 - pi_pred) * sigma, log = TRUE))  
  } else {
    -sum(dnorm(x = pi_obs, mean = pi_pred, sd = sigma, log = TRUE))
  }
  
}

LL_noBGS <- function(pi_obs, par, G_i, fd_i, rbp_i, mode = "beta"){
  
  #rescaled parameters to make optimization easier
  theta <- 1/exp(par[1])
  alpha <- 1/exp(par[2])
  sigma <- 1/exp(par[3])
  
  pi_pred <- theta_noBGS(theta, G_i, alpha, fd_i, rbp_i)
  
  if(mode == "beta"){
    -sum(dbeta(x = pi_obs, shape1 = pi_pred * sigma, shape2 = (1 - pi_pred) * sigma, log = TRUE))  
  } else {
    -sum(dnorm(x = pi_obs, mean = pi_pred, sd = sigma, log = TRUE))
  }
}



LL_noHH <- function(pi_obs, par, G_i, fd_i, rbp_i, mode = "beta"){
  
  #rescaled parameters to make optimization easier
  theta <- 1/exp(par[1])
  sigma <- 1/exp(par[2])
  
  pi_pred <- theta_noHH(theta, G_i, alpha, fd_i, rbp_i)
  
  if(mode == "beta"){
    -sum(dbeta(x = pi_obs, shape1 = pi_pred * sigma, shape2 = (1 - pi_pred) * sigma, log = TRUE))  
  } else {
    -sum(dnorm(x = pi_obs, mean = pi_pred, sd = sigma, log = TRUE))
  }
}


LL_intercept <- function(pi_obs, par, G_i, fd_i, rbp_i, mode = "beta"){
  
  #rescaled parameters to make optimization easier
  theta <- 1/exp(par[1])
  sigma <- 1/exp(par[2])
  
  pi_pred <- theta_intercept(theta, G_i, alpha, fd_i, rbp_i)
  
  if(mode == "beta"){
    -sum(dbeta(x = pi_obs, shape1 = pi_pred * sigma, shape2 = (1 - pi_pred) * sigma, log = TRUE))  
  } else {
    -sum(dnorm(x = pi_obs, mean = pi_pred, sd = sigma, log = TRUE))
  }
}


#get confidence intervals of parameters
get_conf <- function(model, param = c("theta", "alpha", "sigma")){
  
  se <- sqrt(diag(solve(model$hessian)))
  lows <- 1/exp(model$par+1.96*se)
  highs <- 1/exp(model$par-1.96*se)
  expected <- 1/exp(model$par)
  conf_df <- data.frame(
    param = param, 
    lower=lows, 
    value=expected, 
    upper=highs)
  conf_df
}

#prediction intervals based on uncertainty predicted from model
predict_df <- function(model, new_data, E_theta, param = c("theta", "alpha", "sigma"), levels = c(0.025, 0.975), mode = "beta"){
  
  conf_df <- get_conf(model, param = param)
  theta_df <- filter(conf_df, param == "theta")
  alpha_df <- filter(conf_df, param == "alpha")
  sig_df <- filter(conf_df, param == "sigma")
  sigma_low <- sig_df$lower
  sigma_upper <- sig_df$upper
  
  e_exp <- E_theta(theta = theta_df$value[1], 
                   G_i = new_data$G_i, 
                   alpha = alpha_df$value[1], 
                   fd_i = new_data$fd_i, 
                   rbp_i = new_data$rbp_i)
  
  
  if(mode == "beta"){
    pred_df <- 
      e_exp %>%
      map_df(~{
        tibble(
          expected_pi = .x,
          low = quantile(rbeta(100, shape1 = .x*sigma_low, shape2 = (1 - .x) * sigma_low), probs = levels[1]),
          high = quantile(rbeta(100, shape1 = .x*sigma_upper, shape2 = ( 1-.x) * sigma_upper), probs = levels[2])
        )
      })    
  } else {
    pred_df <- 
      e_exp %>%
      map_df(~{
        tibble(
          expected_pi = .x,
          low = quantile(rnorm(100, .x, sigma_low), probs = levels[1]),
          high = quantile(rnorm(100, .x, sigma_upper), probs = levels[2])
        )
      })
  }
  
  bind_cols(new_data, pred_df)
}



AIC_all <- function(mod_full, mod_noBGS, mod_noHH, mod_intercept){
  
  #get model AIC
  AIC <- function(model, K){
    2*K + 2*model$value
  }
  
  model <- c("full", "noBGS", "noHH", "intercept")
  
  nlls <- c("mod_full" = mod_full$value,
            "mod_noBGS" = mod_noBGS$value,
            "mod_noHH" = mod_noHH$value,
            "mod_intercept" = mod_intercept$value)
  
  
  AICs <- c("mod_full" = AIC(mod_full, 3),
            "mod_noBGS" = AIC(mod_noBGS, 3),
            "mod_noHH" = AIC(mod_noHH, 2),
            "mod_intercept" = AIC(mod_intercept, 2))
  
  AIC_delta <- AICs - min(AICs) 
  AIC_weight <- exp(-0.5 * AIC_delta )/sum(exp(-0.5 * AIC_delta ))
  tibble(model, nLL = nlls, AIC = AICs, AIC_delta, AIC_weight)
}


infer_theta <- function(pi, G_i, fd_i, rbp_i, sample_size = 100, pop = "", mode = "beta", conf = TRUE, pred = TRUE){
  
  #model fit
  mod_full <- 
    optim(
      #par = c(2, 2, 2), #initial values of theta, alpha, and sigma
      par = rexp(n = 3, 1/2),
      fn = LL_full, #log likelihood function to be optimized
      pi_obs = pi, G_i = G_i, fd_i = fd_i, rbp_i = rbp_i, #non-optimized parameters passed to LL
      hessian = TRUE, 
      mode = mode
    )
  
  mod_noBGS <- 
    optim(
      #par = c(2, 2, 2), #initial values of theta, alpha, and sigma
      par = rexp(n = 3, 1/2),
      fn = LL_noBGS, #log likelihood function to be optimized
      pi_obs = pi, G_i = G_i, fd_i = fd_i, rbp_i = rbp_i, #non-optimized parameters passed to LL
      hessian = TRUE,
      mode = mode
    )
  
  mod_noHH <- 
    optim(
      #par = c(2,2),
      par = rexp(n = 2, 1/2),
      fn = LL_noHH,
      pi_obs = pi, G_i = G_i, fd_i = fd_i, rbp_i = rbp_i, #non-optimized parameters passed to LL
      hessian = TRUE,
      mode = mode
    )
  
  mod_intercept <- 
    optim(
      #par = c(2, 2),
      par = rexp(n = 2, 1/2),
      fn = LL_intercept,
      pi_obs = pi, G_i = G_i, fd_i = fd_i, rbp_i = rbp_i, #non-optimized parameters passed to LL
      hessian = TRUE,
      mode = mode
    )
  

  aic_df <- AIC_all(mod_full, mod_noBGS, mod_noHH, mod_intercept) %>% 
    mutate(pop = pop, mode = mode)
  
  
  if(conf){
    conf_df <- 
      bind_rows(
        get_conf(mod_full, c("theta", "alpha","sigma")) %>% mutate(model = "full"),
        get_conf(mod_noBGS, c("theta", "alpha","sigma")) %>% mutate(model = "noBGS"),
        get_conf(mod_noHH, c("theta", "sigma")) %>% mutate(model = "noHH"),
        get_conf(mod_intercept, c("theta", "sigma")) %>% mutate(model = "intercept")
      ) %>% 
      mutate(pop = pop, mode = mode)
  } else{conf_df <- tibble()}
  
  #make a data frame of the variables and randomly sample 1000 rows for input
  new_df <- 
    tibble(
      pi = pi,
      G_i = G_i,
      fd_i = fd_i,
      rbp_i = rbp_i 
    ) %>% 
    sample_n(sample_size)
  
    
  if(pred){
  pred_df <- 
    bind_rows(
      predict_df(mod_full, new_data = new_df, param = c("theta","alpha","sigma"), E_theta = theta_full, mode = mode) %>% 
        mutate(model = "full", mode = mode),
      predict_df(mod_intercept, new_data = new_df, param = c("theta","sigma"), E_theta = theta_intercept, mode = mode) %>% 
        mutate(model = "intercept", mode = mode),
      predict_df(mod_noHH, new_data = new_df, param = c("theta","sigma"), E_theta = theta_noHH) %>% 
        mutate(model = "noHH", mode = mode),
      predict_df(mod_noBGS, new_data = new_df, param = c("theta","alpha","sigma"), E_theta = theta_noBGS, mode = mode) %>% 
        mutate(model = "noBGS", mode = mode)
    ) %>% 
    mutate(pop = pop)      
  } else {
    pred_df  <- tibble()
  }  
  
  theta_list <- list(pred_df = pred_df, conf_df = conf_df, aic_df = aic_df)
  return(theta_list)
}


#!!!REQUIRES ALL INPUT DFS INCLUDE COLUMN NAMES chr, start, end
CM <- function(genetic_df, pi_df){
  tibble(
  cm_start = approx(x = genetic_df$pos, y = genetic_df$cm, xout = pi_df$start)$y,
  cm_end = approx(x = genetic_df$pos, y = genetic_df$cm, xout = pi_df$end)$y,
  cm_mid = (cm_start + cm_end)/2,
  cm_mb = approx(x = genetic_df$pos, y = genetic_df$cm_mb, xout = (pi_df$start + pi_df$end)/2)$y,
  rec = cm_end - cm_start
  )
}

#Reading in our PI Data
read_angsd_pi = function(pi_file, minimum_sites = 0.01){
  vroom::vroom(
    pi_file, skip = 1,
    col_names = c("info", "chr", "WinCenter", "tW","tP","tF","tH","tL","Tajima","fuf","fud","fayh","zeng","nSites")) %>%
    separate(info, sep = "[\\)\\(,]", into = c(letters[1:7], "WinStart", "WinStop", "g2")) %>% 
    select(-c(letters[1:7], g2)) %>%
    mutate(start = as.numeric(WinStart), 
           end = as.numeric(WinStop),
           size = end - start) %>% 
    na.omit() %>% 
    mutate(pi = tP / nSites) %>% 
    filter(pi > 0, nSites > minimum_sites*size) %>%
    select(chr, start, end, pi)    
}


#Used to get Exonic Sites per Window -- assumes input from a single chromosome
get_functional_bps <- function(functional_df, pi_df){
  pi_winsize <- unique(pi_df$end[1] - pi_df$start[1]) #assume all windows are the same as first
  maximum_pi_bp <- max(pi_df$end) 
  Exonic_Site_Locations = unique(c(functional_df$start, functional_df$end))
  all_bins <- data.frame(table(cut(Exonic_Site_Locations, seq(1, maximum_pi_bp, pi_winsize-1)))) %>% 
    mutate(Var1 = str_remove_all(string = Var1, pattern = "[\\]\\(]")) %>% 
    separate(Var1, into = c("start", "end"), sep = ",", convert = TRUE) %>% 
    tibble()
  
  left_join(pi_df, all_bins) %>% pull(Freq)
}


build_bgs_fit <- function(chrms, pi_df, genetic_map_df, functional_df, s, h, mu, pop, conf = F, pred = F){
    
    Total_Exonic_Sites <- sum(functional_df$end - functional_df$start)
    
    model_df <- chrms %>%  
      map_df(~{
      pi_winsize <- pi_df$end[1] - pi_df$start[1]
      gen_map_chr <- filter(genetic_map_df, chr == .x)
      pi_df_chr <- filter(pi_df, chr == .x)
      morgan_df <- CM(genetic_df = gen_map_chr, pi_df = pi_df_chr)
      
      gff_chr <- filter(functional_df, chr == .x)
      #functional_bps_raw <- get_functional_bps(functional_df = gff_chr, pi_df = pi_df_chr) 
      
      Exonic_Site_Locations = unique(c(gff_chr$start, gff_chr$end))
      functional_bps_raw <- map_dbl(1:nrow(pi_df_chr), function(x) sum(Exonic_Site_Locations > pi_df_chr$start[x] & Exonic_Site_Locations < pi_df_chr$end[x]))
      functional_bps <- functional_bps_raw/Total_Exonic_Sites    
          
      g_df  <- bind_cols(tibble(functional_bps_raw = functional_bps_raw, functional_bps = functional_bps_raw/Total_Exonic_Sites), morgan_df, pi_df_chr) %>% drop_na()
      functional_bps <- g_df$functional_bps_raw/Total_Exonic_Sites
      G <- get_G(U = mu * Total_Exonic_Sites, sh = s * h, P = 1, fd_i = g_df$functional_bps,  M_1 = g_df$cm_start/100,  M_2 = g_df$cm_end/100)
      Beta = exp(-(mu*g_df$functional_bps_raw)/(g_df$cm_mb/1e4))                            
      rho = (1-exp(-1*(g_df$cm_mb/1e6)*2/100))/2
                                    
      mutate(g_df, G = G, Beta = Beta, rho = rho)
                                    
                                    
      #G <- get_G(U = mu * Total_Exonic_Sites, sh = s * h, P = 1, fd_i = functional_bps,  M_1 = morgan_df$cm_start/100,  M_2 = morgan_df$cm_end/100)
      #Beta = exp(-(mu*functional_bps_raw)/(morgan_df$cm_mb/1e4))
      #rho = (1-exp(-1*(morgan_df$cm_mb/1e6)*2/100))/2
      #tibble(pi, G, functional_bps, rho, Beta, chr = .x, start = pi_df_chr$start, end = pi_df_chr$end)
                                    
    }) %>% 
      drop_na()
    
    mod_fits <- infer_theta(model_df$pi, model_df$G, model_df$functional_bps, model_df$rho, sample_size = nrow(model_df), pop = pop, conf = conf, pred = pred)
    
        list(
        conf_df = mod_fits$conf_df %>% mutate(s = s, mu = mu, h = h, pop = pop),
        pred_df = mod_fits$pred_df %>% mutate(s = s, mu = mu, h = h, pop = pop),
        aic_df = mod_fits$aic_df %>% mutate(s = s, mu = mu, h = h, pop = pop)
        )
    
}

── Attaching packages ─────────────────────────────────────────────────────────────────────────────────────────────────────────────────── tidyverse 1.3.0 ──

✔ ggplot2 3.3.2     ✔ purrr   0.3.4
✔ tibble  3.0.4     ✔ dplyr   1.0.2
✔ tidyr   1.1.2     ✔ stringr 1.4.0
✔ readr   1.3.1     ✔ forcats 0.5.0

── Conflicts ────────────────────────────────────────────────────────────────────────────────────────────────────────────────────── tidyverse_conflicts() ──
✖ dplyr::filter() masks stats::filter()
✖ dplyr::lag()    masks stats::lag()


Attaching package: ‘patchwork’


The following object is masked from ‘package:cowplot’:

    align_plots




In [3]:
pi_files <- list.files(path = "../data/angsd_pi", pattern = "(1000000|100000)BP_theta.thetasWindow.gz.pestPG", full.names = TRUE)
pops <- str_replace(pi_files, "../data/angsd_pi/(..*)_theta.thetasWindow.gz.pestPG", "\\1")

gen_map_all_chr <- read_delim("../data/map/ogut_v5.map.txt", delim = "\t") %>% 
  drop_na() %>%
  mutate(cm = cm + abs(min(cm))) %>%
  group_by(chr) %>% 
  group_modify(~{
    df1 <- slice(.x, -nrow(.x))
    df2 <- slice(.x, -1)
    to_keep <- df2$cm > df1$cm & df2$pos > df1$pos
    df1 <- df1[to_keep, ]
    df2 <- df2[to_keep, ]
    cm_mb <- tibble(cm_mb = 1e6*(df2$cm - df1$cm)/(df2$pos - df1$pos))
    bind_cols(df2, cm_mb)
  }) %>% 
  mutate(chr = paste0("chr", chr))


#made with 
#zcat < data/refs/v5/v5.gff3.gz | grep "P001" | bedtools sort -i stdin | bedtools merge -i stdin > data/refs/v5_functional.bed
gff_test <- vroom::vroom(
  "../data/refs/v5_functional.bed", 
  delim = "\t",
  comment = "#", 
  col_names = c("chr", "start", "end")
  )

chrms <- paste0("chr", 1:10)

pops

Parsed with column specification:
cols(
  chr = col_double(),
  pos = col_double(),
  cm = col_double()
)

Rows: 164,426
Columns: 3
Delimiter: "\t"
chr [1]: chr
dbl [2]: start, end

Use `spec()` to retrieve the guessed column specification
Pass a specification to the `col_types` argument to quiet this message



[1] "v5--LR--Amatlan_de_Canas.1000000BP"     
 [2] "v5--LR--Amatlan_de_Canas.100000BP"      
 [3] "v5--LR--Crucero_Lagunitas.1000000BP"    
 [4] "v5--LR--Crucero_Lagunitas.100000BP"     
 [5] "v5--LR--Los_Guajes.1000000BP"           
 [6] "v5--LR--Los_Guajes.100000BP"            
 [7] "v5--LR--Palmar_Chico.1000000BP"         
 [8] "v5--LR--Palmar_Chico.100000BP"          
 [9] "v5--LR--random.1000000BP"               
[10] "v5--LR--random.100000BP"                
[11] "v5--LR--random1_Palmar_Chico.1000000BP" 
[12] "v5--LR--random1_Palmar_Chico.100000BP"  
[13] "v5--LR--random2_Palmar_Chico.1000000BP" 
[14] "v5--LR--random2_Palmar_Chico.100000BP"  
[15] "v5--LR--San_Lorenzo.1000000BP"          
[16] "v5--LR--San_Lorenzo.100000BP"           
[17] "v5--Teo--Amatlan_de_Canas.1000000BP"    
[18] "v5--Teo--Amatlan_de_Canas.100000BP"     
[19] "v5--Teo--Crucero_Lagunitas.1000000BP"   
[20] "v5--Teo--Crucero_Lagunitas.100000BP"    
[21] "v5--Teo--El_Rodeo.1000000BP"            
[22] "v5--Teo--El_Rodeo.100000BP"             
[23] "v5--Teo--Los_Guajes.1000000BP"          
[24] "v5--Teo--Los_Guajes.100000BP"           
[25] "v5--Teo--Palmar_Chico.1000000BP"        
[26] "v5--Teo--Palmar_Chico.100000BP"         
[27] "v5--Teo--random.1000000BP"              
[28] "v5--Teo--random.100000BP"               
[29] "v5--Teo--random1_Palmar_Chico.1000000BP"
[30] "v5--Teo--random1_Palmar_Chico.100000BP" 
[31] "v5--Teo--random2_Palmar_Chico.1000000BP"
[32] "v5--Teo--random2_Palmar_Chico.100000BP" 
[33] "v5--Teo--San_Lorenzo.1000000BP"         
[34] "v5--Teo--San_Lorenzo.100000BP"

In [ ]:
PI  <-  read_angsd_pi(pi_files[1])
nrow(PI)

In [ ]:
mu=3e-8; s = 1e-4; h = 0.2; chrms = chrms; pi_df = PI; genetic_map_df = gen_map_all_chr; functional_df = gff_test; pop = "test"
Total_Exonic_Sites <- sum(functional_df$end - functional_df$start)

poop <- model_df <- chrms %>%  
  map_df(~{
  .x <- "chr3"    
  pi_winsize <- pi_df$end[1] - pi_df$start[1]
  gen_map_chr <- filter(genetic_map_df, chr == .x)
  pi_df_chr <- filter(pi_df, chr == .x)
  morgan_df <- CM(genetic_df = gen_map_chr, pi_df = pi_df_chr)

  gff_chr <- filter(functional_df, chr == .x)
  #functional_bps_raw <- get_functional_bps(functional_df = gff_chr, pi_df = pi_df_chr) 

  Exonic_Site_Locations = unique(c(gff_chr$start, gff_chr$end))
  functional_bps_raw <- map_dbl(1:nrow(pi_df_chr), function(x) sum(Exonic_Site_Locations > pi_df_chr$start[x] & Exonic_Site_Locations < pi_df_chr$end[x]))
  functional_bps <- functional_bps_raw/Total_Exonic_Sites    
  g_df  <- bind_cols(tibble(functional_bps_raw = functional_bps_raw, functional_bps = functional_bps_raw/Total_Exonic_Sites), morgan_df, pi_df_chr) %>% drop_na()
  functional_bps <- g_df$functional_bps_raw/Total_Exonic_Sites
  G <- get_G(U = mu * Total_Exonic_Sites, sh = s * h, P = 1, fd_i = g_df$functional_bps,  M_1 = g_df$cm_start/100,  M_2 = g_df$cm_end/100)
  Beta = exp(-(mu*g_df$functional_bps_raw)/(g_df$cm_mb/1e4))                            
  rho = (1-exp(-1*(g_df$cm_mb/1e6)*2/100))/2

  mutate(g_df, G = G, Beta = Beta, rho = rho)
 })

In [ ]:
param_grid <- expand_grid(s = 10^seq(-6, -1, length.out = 12), mu = c(3e-8), h = c(.5))
full_fits <- 
pops %>% map(~{
    PI <- read_angsd_pi(str_glue("../data/angsd_pi/{.x}_theta.thetasWindow.gz.pestPG"))
    pmap(param_grid, function(s, mu, h, pop = .x){
        model_df <- build_bgs_fit(
            chrms = chrms, 
            pi_df= PI, 
            genetic_map_df = gen_map_all_chr, 
            functional_df = gff_test, 
            s = s, h = h, mu = mu, pop = .x)
        })    
    })

saveRDS(full_fits, "bgs_fullfit.rds")


Rows: 2,117
Columns: 14
Delimiter: "\t"
chr [ 2]: info, chr
dbl [12]: WinCenter, tW, tP, tF, tH, tL, Tajima, fuf, fud, fayh, zeng, nSites

Use `spec()` to retrieve the guessed column specification
Pass a specification to the `col_types` argument to quiet this message

Warning message in dbeta(x = pi_obs, shape1 = pi_pred * sigma, shape2 = (1 - pi_pred) * :
“NaNs produced”
Warning message in dbeta(x = pi_obs, shape1 = pi_pred * sigma, shape2 = (1 - pi_pred) * :
“NaNs produced”
Warning message in dbeta(x = pi_obs, shape1 = pi_pred * sigma, shape2 = (1 - pi_pred) * :
“NaNs produced”
Warning message in dbeta(x = pi_obs, shape1 = pi_pred * sigma, shape2 = (1 - pi_pred) * :
“NaNs produced”
Warning message in dbeta(x = pi_obs, shape1 = pi_pred * sigma, shape2 = (1 - pi_pred) * :
“NaNs produced”
Warning message in dbeta(x = pi_obs, shape1 = pi_pred * sigma, shape2 = (1 - pi_pred) * :
“NaNs produced”
Warning message in dbeta(x = pi_obs, shape1 = pi_pred * sigma, shape2 = (1 - pi_pred) * :
“NaNs

Warning message in dbeta(x = pi_obs, shape1 = pi_pred * sigma, shape2 = (1 - pi_pred) * :
“NaNs produced”
Warning message in dbeta(x = pi_obs, shape1 = pi_pred * sigma, shape2 = (1 - pi_pred) * :
“NaNs produced”
Warning message in dbeta(x = pi_obs, shape1 = pi_pred * sigma, shape2 = (1 - pi_pred) * :
“NaNs produced”
Warning message in dbeta(x = pi_obs, shape1 = pi_pred * sigma, shape2 = (1 - pi_pred) * :
“NaNs produced”
Warning message in dbeta(x = pi_obs, shape1 = pi_pred * sigma, shape2 = (1 - pi_pred) * :
“NaNs produced”
Warning message in dbeta(x = pi_obs, shape1 = pi_pred * sigma, shape2 = (1 - pi_pred) * :
“NaNs produced”
Warning message in dbeta(x = pi_obs, shape1 = pi_pred * sigma, shape2 = (1 - pi_pred) * :
“NaNs produced”
Warning message in dbeta(x = pi_obs, shape1 = pi_pred * sigma, shape2 = (1 - pi_pred) * :
“NaNs produced”
Rows: 21,296
Columns: 14
Delimiter: "\t"
chr [ 2]: info, chr
dbl [12]: WinCenter, tW, tP, tF, tH, tL, Tajima, fuf, fud, fayh, zeng, nSites

Use `spec()`

Warning message in dbeta(x = pi_obs, shape1 = pi_pred * sigma, shape2 = (1 - pi_pred) * :
“NaNs produced”
Warning message in dbeta(x = pi_obs, shape1 = pi_pred * sigma, shape2 = (1 - pi_pred) * :
“NaNs produced”
Warning message in dbeta(x = pi_obs, shape1 = pi_pred * sigma, shape2 = (1 - pi_pred) * :
“NaNs produced”
Warning message in dbeta(x = pi_obs, shape1 = pi_pred * sigma, shape2 = (1 - pi_pred) * :
“NaNs produced”
Warning message in dbeta(x = pi_obs, shape1 = pi_pred * sigma, shape2 = (1 - pi_pred) * :
“NaNs produced”
Warning message in dbeta(x = pi_obs, shape1 = pi_pred * sigma, shape2 = (1 - pi_pred) * :
“NaNs produced”
Warning message in dbeta(x = pi_obs, shape1 = pi_pred * sigma, shape2 = (1 - pi_pred) * :
“NaNs produced”
Warning message in dbeta(x = pi_obs, shape1 = pi_pred * sigma, shape2 = (1 - pi_pred) * :
“NaNs produced”
Warning message in dbeta(x = pi_obs, shape1 = pi_pred * sigma, shape2 = (1 - pi_pred) * :
“NaNs produced”
Warning message in dbeta(x = pi_obs, shape1 = 

Warning message in dbeta(x = pi_obs, shape1 = pi_pred * sigma, shape2 = (1 - pi_pred) * :
“NaNs produced”
Warning message in dbeta(x = pi_obs, shape1 = pi_pred * sigma, shape2 = (1 - pi_pred) * :
“NaNs produced”
Warning message in dbeta(x = pi_obs, shape1 = pi_pred * sigma, shape2 = (1 - pi_pred) * :
“NaNs produced”
Warning message in dbeta(x = pi_obs, shape1 = pi_pred * sigma, shape2 = (1 - pi_pred) * :
“NaNs produced”
Warning message in dbeta(x = pi_obs, shape1 = pi_pred * sigma, shape2 = (1 - pi_pred) * :
“NaNs produced”
Warning message in dbeta(x = pi_obs, shape1 = pi_pred * sigma, shape2 = (1 - pi_pred) * :
“NaNs produced”
Warning message in dbeta(x = pi_obs, shape1 = pi_pred * sigma, shape2 = (1 - pi_pred) * :
“NaNs produced”
Warning message in dbeta(x = pi_obs, shape1 = pi_pred * sigma, shape2 = (1 - pi_pred) * :
“NaNs produced”
Warning message in dbeta(x = pi_obs, shape1 = pi_pred * sigma, shape2 = (1 - pi_pred) * :
“NaNs produced”
Warning message in dbeta(x = pi_obs, shape1 = 

Warning message in dbeta(x = pi_obs, shape1 = pi_pred * sigma, shape2 = (1 - pi_pred) * :
“NaNs produced”
Warning message in dbeta(x = pi_obs, shape1 = pi_pred * sigma, shape2 = (1 - pi_pred) * :
“NaNs produced”
Warning message in dbeta(x = pi_obs, shape1 = pi_pred * sigma, shape2 = (1 - pi_pred) * :
“NaNs produced”
Warning message in dbeta(x = pi_obs, shape1 = pi_pred * sigma, shape2 = (1 - pi_pred) * :
“NaNs produced”
Warning message in dbeta(x = pi_obs, shape1 = pi_pred * sigma, shape2 = (1 - pi_pred) * :
“NaNs produced”
Warning message in dbeta(x = pi_obs, shape1 = pi_pred * sigma, shape2 = (1 - pi_pred) * :
“NaNs produced”
Warning message in dbeta(x = pi_obs, shape1 = pi_pred * sigma, shape2 = (1 - pi_pred) * :
“NaNs produced”
Warning message in dbeta(x = pi_obs, shape1 = pi_pred * sigma, shape2 = (1 - pi_pred) * :
“NaNs produced”
Warning message in dbeta(x = pi_obs, shape1 = pi_pred * sigma, shape2 = (1 - pi_pred) * :
“NaNs produced”
Warning message in dbeta(x = pi_obs, shape1 = 

...

Warning message in dbeta(x = pi_obs, shape1 = pi_pred * sigma, shape2 = (1 - pi_pred) * :
“NaNs produced”
Warning message in dbeta(x = pi_obs, shape1 = pi_pred * sigma, shape2 = (1 - pi_pred) * :
“NaNs produced”
Warning message in dbeta(x = pi_obs, shape1 = pi_pred * sigma, shape2 = (1 - pi_pred) * :
“NaNs produced”
Warning message in dbeta(x = pi_obs, shape1 = pi_pred * sigma, shape2 = (1 - pi_pred) * :
“NaNs produced”
Warning message in dbeta(x = pi_obs, shape1 = pi_pred * sigma, shape2 = (1 - pi_pred) * :
“NaNs produced”
Warning message in dbeta(x = pi_obs, shape1 = pi_pred * sigma, shape2 = (1 - pi_pred) * :
“NaNs produced”
Warning message in dbeta(x = pi_obs, shape1 = pi_pred * sigma, shape2 = (1 - pi_pred) * :
“NaNs produced”
Warning message in dbeta(x = pi_obs, shape1 = pi_pred * sigma, shape2 = (1 - pi_pred) * :
“NaNs produced”
Warning message in dbeta(x = pi_obs, shape1 = pi_pred * sigma, shape2 = (1 - pi_pred) * :
“NaNs produced”
Warning message in dbeta(x = pi_obs, shape1 = 

Rows: 2,117
Columns: 14
Delimiter: "\t"
chr [ 2]: info, chr
dbl [12]: WinCenter, tW, tP, tF, tH, tL, Tajima, fuf, fud, fayh, zeng, nSites

Use `spec()` to retrieve the guessed column specification
Pass a specification to the `col_types` argument to quiet this message

Warning message in dbeta(x = pi_obs, shape1 = pi_pred * sigma, shape2 = (1 - pi_pred) * :
“NaNs produced”
Warning message in dbeta(x = pi_obs, shape1 = pi_pred * sigma, shape2 = (1 - pi_pred) * :
“NaNs produced”
Warning message in dbeta(x = pi_obs, shape1 = pi_pred * sigma, shape2 = (1 - pi_pred) * :
“NaNs produced”
Warning message in dbeta(x = pi_obs, shape1 = pi_pred * sigma, shape2 = (1 - pi_pred) * :
“NaNs produced”
Warning message in dbeta(x = pi_obs, shape1 = pi_pred * sigma, shape2 = (1 - pi_pred) * :
“NaNs produced”
Warning message in dbeta(x = pi_obs, shape1 = pi_pred * sigma, shape2 = (1 - pi_pred) * :
“NaNs produced”
Warning message in dbeta(x = pi_obs, shape1 = pi_pred * sigma, shape2 = (1 - pi_pred) * :
“NaNs

Warning message in dbeta(x = pi_obs, shape1 = pi_pred * sigma, shape2 = (1 - pi_pred) * :
“NaNs produced”
Warning message in dbeta(x = pi_obs, shape1 = pi_pred * sigma, shape2 = (1 - pi_pred) * :
“NaNs produced”
Warning message in dbeta(x = pi_obs, shape1 = pi_pred * sigma, shape2 = (1 - pi_pred) * :
“NaNs produced”
Warning message in dbeta(x = pi_obs, shape1 = pi_pred * sigma, shape2 = (1 - pi_pred) * :
“NaNs produced”
Warning message in dbeta(x = pi_obs, shape1 = pi_pred * sigma, shape2 = (1 - pi_pred) * :
“NaNs produced”
Warning message in dbeta(x = pi_obs, shape1 = pi_pred * sigma, shape2 = (1 - pi_pred) * :
“NaNs produced”
Warning message in dbeta(x = pi_obs, shape1 = pi_pred * sigma, shape2 = (1 - pi_pred) * :
“NaNs produced”
Warning message in dbeta(x = pi_obs, shape1 = pi_pred * sigma, shape2 = (1 - pi_pred) * :
“NaNs produced”
Warning message in dbeta(x = pi_obs, shape1 = pi_pred * sigma, shape2 = (1 - pi_pred) * :
“NaNs produced”
Warning message in dbeta(x = pi_obs, shape1 = 

Warning message in dbeta(x = pi_obs, shape1 = pi_pred * sigma, shape2 = (1 - pi_pred) * :
“NaNs produced”
Warning message in dbeta(x = pi_obs, shape1 = pi_pred * sigma, shape2 = (1 - pi_pred) * :
“NaNs produced”
Warning message in dbeta(x = pi_obs, shape1 = pi_pred * sigma, shape2 = (1 - pi_pred) * :
“NaNs produced”
Warning message in dbeta(x = pi_obs, shape1 = pi_pred * sigma, shape2 = (1 - pi_pred) * :
“NaNs produced”
Warning message in dbeta(x = pi_obs, shape1 = pi_pred * sigma, shape2 = (1 - pi_pred) * :
“NaNs produced”
Warning message in dbeta(x = pi_obs, shape1 = pi_pred * sigma, shape2 = (1 - pi_pred) * :
“NaNs produced”
Warning message in dbeta(x = pi_obs, shape1 = pi_pred * sigma, shape2 = (1 - pi_pred) * :
“NaNs produced”
Warning message in dbeta(x = pi_obs, shape1 = pi_pred * sigma, shape2 = (1 - pi_pred) * :
“NaNs produced”
Warning message in dbeta(x = pi_obs, shape1 = pi_pred * sigma, shape2 = (1 - pi_pred) * :
“NaNs produced”
Warning message in dbeta(x = pi_obs, shape1 = 

Warning message in dbeta(x = pi_obs, shape1 = pi_pred * sigma, shape2 = (1 - pi_pred) * :
“NaNs produced”
Warning message in dbeta(x = pi_obs, shape1 = pi_pred * sigma, shape2 = (1 - pi_pred) * :
“NaNs produced”
Warning message in dbeta(x = pi_obs, shape1 = pi_pred * sigma, shape2 = (1 - pi_pred) * :
“NaNs produced”
Warning message in dbeta(x = pi_obs, shape1 = pi_pred * sigma, shape2 = (1 - pi_pred) * :
“NaNs produced”
Warning message in dbeta(x = pi_obs, shape1 = pi_pred * sigma, shape2 = (1 - pi_pred) * :
“NaNs produced”
Warning message in dbeta(x = pi_obs, shape1 = pi_pred * sigma, shape2 = (1 - pi_pred) * :
“NaNs produced”
Warning message in dbeta(x = pi_obs, shape1 = pi_pred * sigma, shape2 = (1 - pi_pred) * :
“NaNs produced”
Warning message in dbeta(x = pi_obs, shape1 = pi_pred * sigma, shape2 = (1 - pi_pred) * :
“NaNs produced”
Warning message in dbeta(x = pi_obs, shape1 = pi_pred * sigma, shape2 = (1 - pi_pred) * :
“NaNs produced”
Warning message in dbeta(x = pi_obs, shape1 = 

In [ ]:
#HERE
full_fits <- readRDS("bgs_fullfit.rds")

aic_df <- map_df(full_fits, ~ map_df(.x, function(x) x$aic_df))
                                                                 
best_mod <- aic_df %>% 
    group_by(pop) %>%
    filter(grepl(pattern = "100000BP", pop)) %>% 
    mutate(AIC_delta = AIC - min(AIC), 
           AIC_weight = exp(-0.5 * AIC_delta )/sum(exp(-0.5 * AIC_delta ))) %>% 
           arrange(AIC_delta) %>% slice(1)
                                     
best_mod

In [ ]:
library(patchwork)
chrms <- paste0("chr", 1:10)

best_fit_list <- best_mod %>% 
select(pop, s, mu, h) %>% 
    pmap(function(pop, s, mu, h){
        PI <- read_angsd_pi(str_glue("../data/angsd_pi/{pop}_theta.thetasWindow.gz.pestPG"))
        build_bgs_fit(
            chrms = chrms, 
            pi_df= PI, 
            genetic_map_df = gen_map_all_chr, 
            functional_df = gff_test, 
            s = s, h = h, mu = mu, pop = pop,
            conf = T, pred = T)
        }) 

aic_df <- map_df(best_fit_list, ~ .x$aic_df)
conf_df <- map_df(best_fit_list, ~ .x$conf_df)
pred_df <- map_df(best_fit_list, ~ .x$pred_df) 

In [ ]:
theme_set(cowplot::theme_cowplot(font_size = 10))

a  <- filter(conf_df, model %in% c("full", "intercept"), param != "sigma") %>% 
    mutate(pop = str_remove_all(pop, ".100000BP"),
          pop = str_remove_all(pop, "v5--")) %>% 
    ggplot() +
    geom_segment(mapping = aes(x = pop, xend = pop, y = lower, yend = upper, colour = model)) +
    geom_point(mapping = aes(pop, value, colour = model)) +
    facet_wrap(~param, scales = "free") +
    theme(axis.text.x=element_text(angle = 80, hjust = 1), 
          axis.title.y=element_blank()) +
    xlab("")
    

b  <- best_mod %>% 
    mutate(pop = str_remove_all(pop, ".100000BP"),
           pop = str_remove_all(pop, "v5--"),
          S = "s") %>% 
    ggplot(aes(pop, s)) +
    geom_point() +
    geom_hline(yintercept = 1e-5, lty = 2) +
    facet_wrap(~S, scales = "free") +
    theme(axis.text.x=element_text(angle = 80, hjust = 1),
          axis.title.y=element_blank(),
          legend.position = "n") +
    ylab("") +
    xlab("")

b + a + plot_layout(widths = c(1,2))


In [ ]:
pred_df %>% 
    filter(model == "full") %>% 
    mutate(pop = str_remove_all(pop, ".100000BP"),
           pop = str_remove_all(pop, "v5--")) %>% 
    mutate(rbp_i = -1*(100/2)*log(1 - 2*rbp_i)*1e6) %>% 
    #mutate(rbp_i = log(rbp_i)) %>% 
    ggplot()+
    geom_point(mapping = aes(rbp_i, pi), alpha = 0.2) + 
    #geom_smooth(mapping = aes(rbp_i, pi), se = F, colour = "dodgerblue") +
    geom_line(mapping = aes(rbp_i, expected_pi), colour = "blue") +
    geom_ribbon(mapping = aes(x = rbp_i, ymin = low, ymax =  high), fill = "grey50", alpha = 0.2) +
    #geom_smooth(mapping = aes(x = rbp_i, high), se = F) +
    #geom_smooth(mapping = aes(x = rbp_i, low), se = F) +
    #geom_smooth(mapping = aes(rbp_i, expected_pi), se = F, colour = "dodgerblue") +
    facet_wrap(~pop, scales = "fixed") +
    xlab("log(cm/Mb)") +
    ylab(expression(pi))

In [ ]:
model_df %>% 
  ggplot(aes(-1*(100/2)*log(1 - 2*rho)*1e6, Beta)) +
  geom_point() 

model_theta <- mod1$conf_df %>% filter(model == "full", param == "theta") %>% pull(value)
mean_theta <- mean(model_df$pi)
mod1$conf_df %>% filter(model == "full")

full_tci <- mod1$conf_df %>% filter(model == "full", param == "theta")
full_tci$lower < mean_theta & full_tci$upper > mean_theta  

mod1$pred_df %>%
  filter(model == "full") %>% 
  mutate(rbp_i = -1*(100/2)*log(1 - 2*rbp_i)*1e6) %>% 
  mutate(rbp_i = log(rbp_i)) %>% 
  ggplot()+
  geom_point(mapping = aes(rbp_i, pi)) +
  geom_line(mapping = aes(rbp_i, expected_pi), colour = "blue") +
  geom_smooth(mapping = aes(rbp_i, expected_pi), se = F, colour = "dodgerblue") +
  geom_ribbon(mapping = aes(x = rbp_i, ymin = low, ymax =  high), fill = "grey50", alpha = 0.2) +
  geom_hline(yintercept = mean_theta, colour = "green") +
  geom_hline(yintercept = model_theta, colour = "blue") 

mod1


In [ ]:
PI = read_angsd_pi(pi_file = "../data/angsd_pi/v5--LR--Amatlan_de_Canas.100000BP_theta.thetasWindow.gz.pestPG") %>% 
    mutate(chr = str_remove_all(chr, "chr"))



In [ ]:
#JUST BETA on v4 for Mitra

library(tidyverse)
library(Rcpp)

#!!!REQUIRES ALL INPUT DFS INCLUDE COLUMN NAMES chr, start, end

#Calculates G
cppFunction(
  'NumericVector get_G(double U, double sh, double P, NumericVector fd_i, NumericVector M_1, NumericVector M_2) {

  int n = M_1.size();
  double G_k;
  double M_i;
  NumericVector G_i(n);

  for(int i = 0; i < n; ++i) {
    M_i =  (M_1[i] + M_2[i]) / 2;
    for(int k = 0; k < n; ++k){
      G_k = U * fd_i[k] * sh / (2 * (sh + P*std::abs(M_i - M_1[k]))*(sh + P*std::abs(M_i - M_2[k])));
      G_i[i] += G_k;
    }
  }
  return G_i;
}'
)


CM <- function(genetic_df, pi_df){
  tibble(
  cm_start = approx(x = genetic_df$pos, y = genetic_df$cm, xout = pi_df$start)$y,
  cm_end = approx(x = genetic_df$pos, y = genetic_df$cm, xout = pi_df$end)$y,
  cm_mid = (cm_start + cm_end)/2,
  cm_mb = approx(x = genetic_df$pos, y = genetic_df$cm_mb, xout = (pi_df$start + pi_df$end)/2)$y,
  rec = cm_end - cm_start
  )
}


est_beta <- function(mu, s = 1e-4, h = 0.2, chrms, pi_df, genetic_map_df, functional_df, pop){
    
    Total_Exonic_Sites <- sum(functional_df$end - functional_df$start) 
    model_df <- chrms %>%  
    map_df(~{
        pi_winsize <- pi_df$end[1] - pi_df$start[1]
        gen_map_chr <- filter(genetic_map_df, chr == .x)
        pi_df_chr <- filter(pi_df, chr == .x)
        morgan_df <- CM(genetic_df = gen_map_chr, pi_df = pi_df_chr)
        gff_chr <- filter(functional_df, chr == .x)
        Exonic_Site_Locations = unique(c(gff_chr$start, gff_chr$end))
        functional_bps_raw <- map_dbl(1:nrow(pi_df_chr), function(x) sum(Exonic_Site_Locations > pi_df_chr$start[x] & Exonic_Site_Locations < pi_df_chr$end[x]))
        g_df <- bind_cols(tibble(functional_bps_raw = functional_bps_raw, functional_bps = functional_bps_raw/Total_Exonic_Sites), morgan_df, pi_df_chr) %>% drop_na()
        functional_bps <- g_df$functional_bps_raw/Total_Exonic_Sites
        G <- get_G(U = mu * Total_Exonic_Sites, sh = s * h, P = 1, fd_i = g_df$functional_bps,  M_1 = g_df$cm_start/100,  M_2 = g_df$cm_end/100)
        Beta = exp(-(mu*g_df$functional_bps_raw)/(g_df$cm_mb/1e4))
        mutate(g_df, G = G, Beta = Beta)
    }) 
}


ogut_path = "../../parv_local/shic/data/ogut_fifthcM_map_agpv4_INCLUDE.txt"
gen_map_all_chr <- read_delim(ogut_path, delim = "\t",
                             col_names = c("one", "two", "cm", "chr", "pos")) %>% 
  drop_na() %>%
  mutate(cm = cm + abs(min(cm))) %>%
  group_by(chr) %>% 
  group_modify(~{
    df1 <- slice(.x, -nrow(.x))
    df2 <- slice(.x, -1)
    to_keep <- df2$cm > df1$cm & df2$pos > df1$pos
    df1 <- df1[to_keep, ]
    df2 <- df2[to_keep, ]
    cm_mb <- tibble(cm_mb = 1e6*(df2$cm - df1$cm)/(df2$pos - df1$pos))
    bind_cols(df2, cm_mb)
  }) 

#made with 
#cat ../parv_local/adapt_mode/data/Zea_mays.B73_RefGen_v4.45.gff3  | grep "P001" | bedtools sort -i stdin | bedtools merge -i stdin > ~/v4_functional_bps.bed
gff_test <- vroom::vroom(
  "~/v4_functional_bps.bed", 
  delim = "\t",
  comment = "#", 
  col_names = c("chr", "start", "end")
  ) %>% 
drop_na()

chrms <- paste0(1:10)
PI = vroom::vroom("~/v4_steps.bed", "\t")
beta_df = est_beta(3e-8, chrms = chrms, pi_df = PI, genetic_map_df = gen_map_all_chr, functional_df = gff_test) #%>% drop_na()
       

In [ ]:
beta_df %>% 
    ggplot(aes((start+end)/2, G)) +
    geom_line() +
    facet_wrap(~chr)

beta_df %>% 
    ggplot(aes((start+end)/2, Beta)) +
    geom_line() +
    facet_wrap(~chr)


write_csv(beta_df, path = "~/v4_G_Beta.csv")


In [ ]:
head(beta_df)

In [ ]:
#pi_winsize <- unique(pi_df$end[1] - pi_df$start[1]) #assume all windows are the same as first
#maximum_pi_bp <- max(pi_df$end)
#Exonic_Site_Locations = unique(c(functional_df$start, functional_df$end))
#all_bins <- data.frame(table(cut(Exonic_Site_Locations, seq(1, maximum_pi_bp, pi_winsize-1)))) %>% 
#mutate(Var1 = str_remove_all(string = Var1, pattern = "[\\]\\(]")) %>% 
#separate(Var1, into = c("start", "end"), sep = ",", convert = TRUE)

map_dbl(1:nrow(pi_df), ~sum(Exonic_Site_Locations > pi_df$start[.x] & Exonic_Site_Locations < pi_df$end[.x]))

In [ ]:
#mu=3e-8; s = 1e-4; h = 0.2; chrms = chrms;, pi_df = PIl genetic_map_df = gen_map_all_chr; functional_df = gff_test; pop = "test"
(Total_Exonic_Sites <- sum(functional_df$end - functional_df$start) )

.x
pi_winsize <- pi_df$end[1] - pi_df$start[1]
gen_map_chr <- filter(genetic_map_df, chr == .x)
pi_df_chr <- filter(pi_df, chr == .x)
morgan_df <- CM(genetic_df = gen_map_chr, pi_df = pi_df_chr)
gff_chr <- filter(functional_df, chr == .x)
Exonic_Site_Locations = unique(c(gff_chr$start, gff_chr$end))
functional_bps_raw <- map_dbl(1:nrow(pi_df_chr), function(x) sum(Exonic_Site_Locations > pi_df_chr$start[x] & Exonic_Site_Locations < pi_df_chr$end[x]))
g_df  <- bind_cols(tibble(functional_bps_raw = functional_bps_raw, functional_bps = functional_bps_raw/Total_Exonic_Sites), morgan_df, pi_df_chr) %>% drop_na()
functional_bps <- g_df$functional_bps_raw/Total_Exonic_Sites
G <- get_G(U = mu * Total_Exonic_Sites, sh = s * h, P = 1, fd_i = g_df$functional_bps,  M_1 = g_df$cm_start/100,  M_2 = g_df$cm_end/100)
Beta = exp(-(mu*g_df$functional_bps_raw)/(g_df$cm_mb/1e4))
mutate(g_df, G = G, Beta = Beta)